<a href="https://colab.research.google.com/github/efeecllk/investor-sentiments/blob/main/bitcoin_tweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install emoji

In [31]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import gensim
from sklearn.linear_model import LogisticRegression
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import string
import re
import emoji


In [32]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [33]:
# Load Bitcoin Tweet Dataset
tweets_df = pd.read_csv('/content/drive/MyDrive/NLP/Bitcoin_tweets.csv', nrows=10000)

# Load Bitcoin Price Dataset
price_df = pd.read_csv('/content/drive/MyDrive/NLP/bitcoin_2017_to_2023.csv')



In [34]:
tweets_df.head()


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [35]:
price_df.head()

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,2023-08-01 13:19:00,28902.48,28902.49,28902.48,28902.49,4.68658,1.354538e+05,258,0.89391,25836.224836
1,2023-08-01 13:18:00,28902.48,28902.49,28902.48,28902.49,4.77589,1.380351e+05,317,2.24546,64899.385195
2,2023-08-01 13:17:00,28908.52,28908.53,28902.48,28902.49,11.52263,3.330532e+05,451,2.70873,78290.170121
3,2023-08-01 13:16:00,28907.41,28912.74,28907.41,28908.53,15.89610,4.595556e+05,483,10.22981,295738.166916
4,2023-08-01 13:15:00,28896.00,28907.42,28893.03,28907.41,37.74657,1.090761e+06,686,16.50452,476955.246611


In [36]:
import pandas as pd

# Convert dates and times
tweets_df['user_created'] = pd.to_datetime(tweets_df['user_created'], errors='coerce')
price_df['timestamp'] = pd.to_datetime(price_df['timestamp'], errors='coerce')

# Clean invalid values and sort by time
tweets_df = tweets_df.dropna(subset=['user_created']).sort_values(by='user_created')
price_df = price_df.dropna(subset=['timestamp']).sort_values(by='timestamp')

# Match timestamps while preserving all tweets
merged_df = pd.merge_asof(
    tweets_df,
    price_df,
    left_on='user_created',
    right_on='timestamp',
    direction='backward',
    tolerance=pd.Timedelta('1d')  # Maximum 1-day difference
)

# Calculate time difference (optional)
merged_df['time_difference'] = (merged_df['user_created'] - merged_df['timestamp']).abs()

# Check for unmatched rows
missing_matches = merged_df[merged_df['timestamp'].isna()]
print(f"Number of Unmatched Tweets: {len(missing_matches)}")

# Perform forward search for unmatched tweets
merged_df_forward = pd.merge_asof(
    tweets_df,
    price_df,
    left_on='user_created',
    right_on='timestamp',
    direction='forward',
    tolerance=pd.Timedelta('1d')
)

# Combine and remove duplicates
final_merged_df = pd.concat([merged_df, merged_df_forward]).drop_duplicates(subset=['user_created'])

# Check still unmatched rows
still_missing = final_merged_df[final_merged_df['timestamp'].isna()]
print(f"Number of Still Unmatched Tweets: {len(still_missing)}")

# Display results
print("\nMatched Tweet and Price Data:")
print(final_merged_df.head())

# Remove unmatched records (rows where 'timestamp' is NaN)
final_merged_df = final_merged_df.dropna(subset=['timestamp']).reset_index(drop=True)

# Verify the final dataset
print(f"Number of Unmatched Tweets After Removal: {final_merged_df['timestamp'].isna().sum()}")
print("\nMatched Tweet and Price Data (Unmatched Removed):")
print(final_merged_df.head())


Number of Unmatched Tweets: 3969
Number of Still Unmatched Tweets: 1802

Matched Tweet and Price Data:
        user_name         user_location  \
0       Greg Kidd       Crystal Bay, NV   
1     Yaron Samid              Tel Aviv   
2  Travis Russell                   NaN   
3   ngc_3370 LION       ngc_3370 Galaxy   
4  Justin Roberti  Nonspecificville, PA   

                                    user_description        user_created  \
0  CEO GlobaliD (@myglobal_id), co-founder Hard Y... 2006-04-01 03:32:37   
1  3X dad, 3X founder, investor and community bui... 2006-12-04 19:48:28   
2  #Bitcoin | Aquaponics | Padres | https://t.co/... 2007-01-26 19:40:36   
3  “As soon as you stop wanting or dreaming somet... 2007-03-12 07:41:16   
4  Writer & podcast producer - Games, tech, finte... 2007-04-03 01:57:06   

   user_followers  user_friends  user_favourites  user_verified  \
0         12868.0           132              417          False   
1          4792.0           140             124

In [37]:
# Remove unmatched records (rows where 'timestamp' is NaN)
final_merged_df = final_merged_df.dropna(subset=['timestamp']).reset_index(drop=True)

# Verify the final dataset
print(f"Number of Unmatched Tweets After Removal: {final_merged_df['timestamp'].isna().sum()}")
print("\nMatched Tweet and Price Data (Unmatched Removed):")
print(final_merged_df.head())
merged_df = final_merged_df

Number of Unmatched Tweets After Removal: 0

Matched Tweet and Price Data (Unmatched Removed):
        user_name        user_location  \
0  Michael Arouet  Somewhere in Europe   
1       athcrypt℗              Croatia   
2     Wickhunterr                India   
3           Kayth               Europe   
4  The 1 i Am Neo      Los Angeles, CA   

                                    user_description        user_created  \
0  Independent investor, correlation deviations b... 2017-08-17 11:22:22   
1  MIT / UX & Product Designer / $BTC $ETH $DOGE ... 2017-08-18 09:29:07   
2  Pure Technical Analyst 🔥\nTrading Crypto since... 2017-08-18 14:19:50   
3                  Software Engineer. #Bitcoin only. 2017-08-18 16:40:28   
4                                     Bitcoin Trader 2017-08-18 17:36:59   

   user_followers  user_friends  user_favourites  user_verified  \
0           795.0           482             1530          False   
1           413.0           396            23026          Fal

In [38]:
merged_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,...,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,time_difference
0,Michael Arouet,Somewhere in Europe,"Independent investor, correlation deviations b...",2017-08-17 11:22:22,795.0,482,1530,False,2021-02-09 22:34:39,"#Bitcoin unrealized losses must be recognized,...",...,4474.80,4474.80,4474.80,4474.80,0.000000,0.000000,0.0,0.000000,0.000000,0 days 00:00:22
1,athcrypt℗,Croatia,MIT / UX & Product Designer / $BTC $ETH $DOGE ...,2017-08-18 09:29:07,413.0,396,23026,False,2021-02-09 07:20:41,Hacked with nudes #blockfolio @blockfolio #btc...,...,4321.20,4327.25,4307.01,4307.01,0.369231,1595.893279,7.0,0.086659,374.904408,0 days 00:00:07
2,Wickhunterr,India,Pure Technical Analyst 🔥\nTrading Crypto since...,2017-08-18 14:19:50,80.0,167,1356,False,2021-02-10 00:05:25,The So called Next BTC is waking up\n\n#YFI #D...,...,4272.92,4272.92,4272.92,4272.92,0.107711,460.240486,1.0,0.107711,460.240486,0 days 00:00:50
3,Kayth,Europe,Software Engineer. #Bitcoin only.,2017-08-18 16:40:28,40.0,428,1628,False,2021-02-08 19:07:28,If you want to create a bitcoin seed with dice...,...,4266.72,4266.72,4266.72,4266.72,0.060301,257.287483,1.0,0.060301,257.287483,0 days 00:00:28
4,The 1 i Am Neo,"Los Angeles, CA",Bitcoin Trader,2017-08-18 17:36:59,142.0,95,3,False,2021-02-08 16:37:20,#Bitcoin #BTC #btcusd $btc https://t.co/lAMPlm...,...,4126.45,4126.45,4126.45,4126.45,0.000000,0.000000,0.0,0.000000,0.000000,0 days 00:00:59


In [39]:
merged_df = merged_df.drop('user_location', axis=1)

In [40]:
merged_df.columns

Index(['user_name', 'user_description', 'user_created', 'user_followers',
       'user_friends', 'user_favourites', 'user_verified', 'date', 'text',
       'hashtags', 'source', 'is_retweet', 'timestamp', 'open', 'high', 'low',
       'close', 'volume', 'quote_asset_volume', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume',
       'time_difference'],
      dtype='object')

In [41]:
merged_df.shape[0]

2415

In [42]:
def preprocess_text(text):
    text = emoji.demojize(text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text)])
    return text



In [43]:
len(merged_df)

2415

In [44]:
merged_df['text'] = merged_df['text'].apply(preprocess_text)


In [45]:
merged_df.head()

,user_name,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,...,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,time_difference
0,Michael Arouet,"Independent investor, correlation deviations b...",2017-08-17 11:22:22,795.0,482,1530,False,2021-02-09 22:34:39,bitcoin unrealized loss must recognized unreal...,"['Bitcoin', 'BTC']",...,4474.80,4474.80,4474.80,4474.80,0.000000,0.000000,0.0,0.000000,0.000000,0 days 00:00:22
1,athcrypt℗,MIT / UX & Product Designer / $BTC $ETH $DOGE ...,2017-08-18 09:29:07,413.0,396,23026,False,2021-02-09 07:20:41,hacked nude blockfolio blockfolio btc bitcoin ...,"['blockfolio', 'btc', 'bitcoin', 'ethereum']",...,4321.20,4327.25,4307.01,4307.01,0.369231,1595.893279,7.0,0.086659,374.904408,0 days 00:00:07
2,Wickhunterr,Pure Technical Analyst 🔥\nTrading Crypto since...,2017-08-18 14:19:50,80.0,167,1356,False,2021-02-10 00:05:25,called next btc waking yfi defi btc binance bi...,"['YFI', 'DeFi', 'BTC', 'Binance', 'Bitcoin', '...",...,4272.92,4272.92,4272.92,4272.92,0.107711,460.240486,1.0,0.107711,460.240486,0 days 00:00:50
3,Kayth,Software Engineer. #Bitcoin only.,2017-08-18 16:40:28,40.0,428,1628,False,2021-02-08 19:07:28,want create bitcoin seed dice roll time roll d...,NaN,...,4266.72,4266.72,4266.72,4266.72,0.060301,257.287483,1.0,0.060301,257.287483,0 days 00:00:28
4,The 1 i Am Neo,Bitcoin Trader,2017-08-18 17:36:59,142.0,95,3,False,2021-02-08 16:37:20,bitcoin btc btcusd btc httpstcolamplmodt live ...,"['Bitcoin', 'BTC', 'btcusd']",...,4126.45,4126.45,4126.45,4126.45,0.000000,0.000000,0.0,0.000000,0.000000,0 days 00:00:59


In [58]:
with open('/content/drive/MyDrive/NLP/word2vec_model.bin', 'rb') as file:
    word2vec_model = pickle.load(file)


with open('/content/drive/MyDrive/NLP/word2vec_logistic_model.pkl', 'wb') as file:
    pickle.dump(sentiment_classifier, file, protocol=2, fix_imports=True) # protocol=2 can offer more backwards compatibility

def text_to_word2vec(text, model):
    words = text.split()
    word_vectors = [model[word] for word in words if word in model.key_to_index]

    if len(word_vectors) == 0:
        # Return a zero vector if no valid words are found
        return np.zeros(model.vector_size)
    else:
        return np.mean(word_vectors, axis=0)

# Sentiment Prediction Function
def sentiment_analysis(text):
    vector = text_to_word2vec(text, word2vec_model)
    sentiment = sentiment_classifier.predict([vector])[0]
    confidence = max(sentiment_classifier.predict_proba([vector])[0])
    return sentiment, round(confidence, 2)

# Apply Sentiment Analysis on Tweets
merged_df['sentiment_label'], merged_df['confidence_score'] = zip(
    *merged_df['text'].apply(lambda x: sentiment_analysis(x))
)

# Display sample rows with sentiment labels
print("\nDataset with Sentiment Labels:")
print(merged_df[['text', 'sentiment_label', 'confidence_score']].head())


NameError: name 'sentiment_classifier' is not defined